In [1]:
require(tidyverse)
require(rstan)
require(ggplot2)
require(mvtnorm)
rstan_options(auto_write = TRUE)
require(bayesplot)
missing_val=-100

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: rstan

Loading required package: StanHeaders

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Attaching package: ‘rstan’


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: mvtnorm

Loading required package: bayesplot

This is bayesplot version 1.7.1

- Online documentation and v

In [3]:
path = file.path('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis', '/20200319_2_stage_impute_beta_missing')
dir.create(path)
setwd(path)

Warning message in dir.create(path):
“'/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis//20200319_2_stage_impute_beta_missing' already exists”


In [4]:
df_nmr = read_csv('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/11_20_2019_even_odds_interaction_prior_1/nmr_metabolites_scaled_matrix.csv')
metabolites = colnames(df_nmr)[1:27]

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [5]:
#TARGET_MISSING_RATE=0.3
get_alpha_beta_for_target_rate = function(df,metabolites,TARGET_MISSING_RATE) {
    missing_rate = df_nmr[,metabolites] %>% summarise_all(function(x) { mean(x == min(x))  })
    missing_rate_numeric = missing_rate %>% t() %>% as.numeric
    mu = mean( missing_rate_numeric )
    sigma_2 = var(missing_rate_numeric)
    RATE = TARGET_MISSING_RATE/mu
    mu = RATE * mu
    sigma_2 = RATE * sigma_2
    mu
    sigma_2
    nu = mu * (1 - mu) / sigma_2 - 1
    alpha = mu * nu
    beta = (1 - mu) * nu
    list(alpha=alpha,beta=beta,mu=mu,sigma_2=sigma_2)
}

get_alpha_beta_for_target_rate(df_nmr,metabolites,0.3)

$alpha
[1] 2.435085

$beta
[1] 5.681866

$mu
[1] 0.3

$sigma_2
[1] 0.02303402

In [6]:
died = df_nmr$died_90_day
sigma_0 = cov(df_nmr[died == 0,c(1:27)])
mu_0 = colMeans(df_nmr[died == 0,c(1:27)]) 
sigma_1 = cov(df_nmr[died == 1,c(1:27)])
mu_1 = colMeans(df_nmr[died == 1,c(1:27)]) 

### Generation code

In [7]:
make_df = function(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.5,censor=TRUE,max_missing=0.6,alpha=-1,beta=-1) {
    diffs = abs( mu_1 - mu_0) / sqrt( (diag(sigma_0)^2)/n_0 + (diag(sigma_1)^2)/n_1  )
    #print(diffs)
    m = length(diffs)
    diffs = diffs/sum(diffs)
    #print(diffs)
    ind = seq(1,m)
    set_non_zero = sample(ind,size=ceiling(frac_sig * m),prob=diffs)
    set_zero = setdiff(ind,set_non_zero)
    truth = rep(FALSE,m)
    truth[set_non_zero] = TRUE
    mu_0[set_zero] = 0
    mu_1[set_zero] = 0
    X_0 = rmvnorm(n = n_0, mean = mu_0, sigma = sigma_0) 
    X_1 = rmvnorm(n = n_1, mean = mu_1, sigma = sigma_1)
    df_sample = data.frame(rbind(X_0,X_1))
    df_sample$died_90_day = c( rep(0,n_0),rep(1,n_1)  )
    
    df_censored = df_sample
    df_naive_impute = df_sample
    n_var = length(mu_0)
    thresholds = rep(0,n_var)
    est.thresholds = rep(0,n_var)
    est.naive_impute = rep(0,n_var)
    missing_rates=rep(0,n_var)
    if(censor == TRUE) {
        
        missing_rates = rbeta(n_var,shape1 = alpha, shape2 = beta)
        # only allow a missing rate up to a certain level
        missing_rates = sapply(missing_rates, function(x) { min(max_missing,x) })
        
        #missing_rates = rep(MISSING_RATE, n_var)
        
        for(i in 1:n_var) {
            thresholds[i] = quantile( df_censored[, i], missing_rates[i] )
            df_censored[ (df_censored[, i] < thresholds[i]), i ] = NA
            est.thresholds[i] = min( df_censored[ , i ], na.rm = T)
            est.naive_impute[i] = log( min( exp(df_censored[ , i ]), na.rm = T) / 2 )
            df_naive_impute[ (df_naive_impute[, i] < thresholds[i]), i ] = est.thresholds[i]
        }
    }
    
    
    list(df_sample=df_sample,truth=truth, df_censored=df_censored, df_naive_impute=df_naive_impute, missing_rates=missing_rates, thresholds=thresholds, est.thresholds=est.thresholds, est.naive_impute=est.naive_impute)
}

### Stage 1 Imputation model

In [8]:
stage_1_impute = "
functions {
    real normal_ub_rng(real mu, real sigma, real ub) {
      real p_ub = normal_cdf(ub, mu, sigma);
      real u = uniform_rng(0, p_ub);
      real y = mu + sigma * inv_Phi(u);
      return y;
    }
}
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;
    int<lower=0> N_use_impute;

    matrix[ N, N_var ] x_naive_impute;
    matrix[ N, N_var ] x_is_missing;

    //contains the indicies of x_naive_imp to use for imputation
    int x_use_impute[N_use_impute, N_var ];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    

    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    
}
parameters {

    //betas for each imputed value
    matrix[ N_use_impute, N_var ] beta_impute_raw;
    vector[ N_var ] alpha_impute;
    vector<lower=0>[ N_var ] sigma_impute;
    real<lower=0> sigma_beta_impute_x;

}
transformed parameters {
    matrix[ N_use_impute, N_var ] beta_impute;
    for(j in 1:N_var) {
        beta_impute[,j] = beta_impute_raw[,j] * sigma_beta_impute_x;
    }
}
model {
    sigma_beta_impute_x ~ std_normal();

    for(j in 1:N_var) {
        real y[N];
        vector[N_use_impute] beta_x_pred;
        int x_use_ind[N_use_impute];
        matrix[N, N_use_impute] x_pred_use;
        real alpha_x_pred;
        real sigma_x_pred;
        //predicted target
        vector[N] xj_pred;
        //target
        vector[N] xj;
        int observed[N];
        
        //ADD TO LIKELIHOOD
        beta_impute_raw[, j] ~ std_normal();
    
        //ASSIGN VALS
        xj = x_naive_impute[, j];
        //dont make a prediction using itself
        //this should be copying the values according to ref
        beta_x_pred = beta_impute[, j];
        alpha_x_pred = alpha_impute[j];
        x_use_ind = x_use_impute[, j];
        sigma_x_pred = sigma_impute[j];
        x_pred_use = x_naive_impute[, x_use_ind];
        
        //Make pred
        xj_pred = x_pred_use * beta_x_pred + alpha_x_pred;
        
        
        //observed = (x_is_missing[, j] == 0);
        
        // target +=  normal_lpdf(xj | xj_pred, sigma_x_pred);

        for(i in 1:N) {
            real mu_x_pred;
            mu_x_pred = xj_pred[i];
            if( x_is_missing[i,j] == 0) {
                // if observed use pdf for observed value
                target +=  normal_lpdf(xj[i] | mu_x_pred, sigma_x_pred);
            } else {
                // if censored, use normal CDF
                target +=  normal_lcdf(threshold[j] | mu_x_pred, sigma_x_pred);
            }
        }
    }
}
generated quantities {
    real x_impute_mean[ N_miss ];
    real x_impute[ N_miss ];

    for(i in 1:N_miss) {
        //These correspond to the rows and columns in the original data frame
        int metab_row;
        int metab_col;
        row_vector[N_use_impute] x_pred_use;
        vector[N_use_impute] beta_x_pred;
        real alpha_x_pred;
        int x_use_ind[N_use_impute];
        real mu_x;
        real sd_x;
        real ub_x;

        metab_row = x_missing[i,1];
        metab_col = x_missing[i,2];
        x_use_ind = x_use_impute[, metab_col ];
        beta_x_pred = beta_impute[, metab_col ];
        alpha_x_pred = alpha_impute[ metab_col ];
        x_pred_use = x_naive_impute[metab_row, x_use_ind];

        mu_x = x_pred_use * beta_x_pred + alpha_x_pred;
        sd_x =  sigma_impute[ metab_col ];
        ub_x = threshold[ metab_col ];
        x_impute_mean[ i ] = mu_x;
        x_impute[i] = normal_ub_rng(mu_x, sd_x, ub_x);
    }
}

"

In [9]:
stage_1_impute_code_file = 'stage_1_impute.stan'
write_lines(x = stage_1_impute, path = stage_1_impute_code_file)
stage_1_impute_model = stan_model(file = stage_1_impute_code_file, verbose = FALSE)

In [10]:
stage_1_impute_get_init_vals = function(data) {
    n_var = data$N_var
    N_use_impute = data$N_use_impute
    # matrix[ N_use_impute, N_var ] beta_impute_raw;
    beta_impute_raw = matrix( rep(0, data$N_use_impute * n_var), ncol=n_var)
    alpha_impute = rep(0,n_var)
    x_use_impute = data$x_use_impute
    sigma_impute = rep(1, n_var)
    
    for(i in 1:n_var) {
        x_imp_vals_use = data$x_naive_impute[, x_use_impute[,i]   ]
        y_out_imp = data$x_naive_impute[,i]
        m_out = glm(y_out_imp ~ x_imp_vals_use,family="gaussian")
        coefs_imp = summary(m_out)$coef
        alpha_impute[i] = coefs_imp[1,1]
        for(k in 1:N_use_impute) {
            beta_impute_raw[k,i] = coefs_imp[1 + k,1]
        }
        sigma_impute[i] = sum(m_out$residuals^2) / m_out$df.residual
    }
    
    sigma_beta_impute_x = sqrt( sum( beta_impute_raw^2 ) / (prod(dim(beta_impute_raw)) - 1) ) 
    
    inits = list(
        sigma_impute = sigma_impute,
        sigma_beta_impute_x = sigma_beta_impute_x,
        alpha_impute = alpha_impute,
        beta_impute_raw = beta_impute_raw / sigma_beta_impute_x
    )
    
    inits
    
}

In [11]:
stage_1_impute_make_data = function(x_censored, y, threshold=NULL,
                                         N_use_impute=8) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
    x_is_missing = apply( is.na(x_censored), FUN=as.numeric, MARGIN=c(1,2))             
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            
    # create a matrix that contains the N_use_impute indicies of the most correlated
    # variables with the metabolite of interest
    # each column contains a metabolite corresponding with x_censored
    # each row corresponds to the index of the metabolite most correlated with it up to N_use_impute
    # rows
    var_cors = cor(x_naive_impute,use = "complete")
    diag(var_cors) = 0
    get_inds_to_use_per_metabolite = function(x) { which( rank(-abs(x)) <= N_use_impute)   }
    x_use_impute = apply(var_cors,MARGIN=1,FUN=get_inds_to_use_per_metabolite )

    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    if( N_miss > 0 ) {
        # make a vector with naive imputation values
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    }

    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            N_use_impute = N_use_impute,
            x_use_impute = x_use_impute,
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_is_missing = x_is_missing,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [12]:
replicate_init = function(init_data,chains=4) {
    init = list()
    for(i in 1:chains){
        init[paste0(i)] = list(init_data)
    }
    init
} 

In [34]:
set.seed(0)
n_0=100
n_1=100
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = 0.3)
#out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.4,censor=TRUE,
#                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)

out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.1,censor=FALSE,
                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)


write_rds(out,'debug_df.rds')

x_censored = out$df_censored[,metabolites]
y=out$df_censored$died_90_day
sum(is.na(x_censored))

stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=1
    )
str(stage_1_data)

[1] 0

List of 13
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] 0.347 0.223 1.455 0.363 0.717 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] 0.347 0.223 1.455 0.363 0.717 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ N_use_impute      : num 8
 $ x_use_impute      : int [1:8, 1:27] 2 15 17 18 19 20 26 27 1 4 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_is_missing      : num [1:200, 1:27] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. 

In [14]:
stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=1, chains=1, iter = 2000, control=list(adapt_delta=0.8))



SAMPLING FOR MODEL 'stage_1_impute' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.02 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 200 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 89.57 seconds (Warm-up)
Chain 1:                59.53 seconds (Sampling)
Chain 1:                149.

In [15]:
df_sum = data.frame(summary(stage_1_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_1_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(10)
df_sum %>% arrange(-abs(Rhat)) %>% head(10)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-5512.6520737,1.1956927349,16.06832048,-5.543797e+03,-5.523679e+03,-5513.1013050,-5501.2417105,-5480.7377332,180.5934,0.9995295,lp__
2,0.1972392,0.0006506298,0.01056270,1.780939e-01,1.895429e-01,0.1964747,0.2043881,0.2188030,263.5613,0.9991405,sigma_beta_impute_x
3,3.6329327,0.0134496092,0.27568563,3.097374e+00,3.445525e+00,3.6250536,3.8110687,4.1945714,420.1544,0.9991785,"beta_impute_raw[7,16]"
4,4.3375261,0.0148969168,0.31698955,3.720797e+00,4.117323e+00,4.3296303,4.5447819,4.9611346,452.7903,0.9990645,"beta_impute_raw[7,25]"
5,3.3370878,0.0127093304,0.28440287,2.791659e+00,3.141564e+00,3.3299810,3.5263381,3.9184802,500.7519,0.9997764,"beta_impute_raw[8,17]"
6,3.4715000,0.0103898173,0.31538750,2.843631e+00,3.245802e+00,3.4727668,3.6934118,4.0503065,921.4529,0.9993288,"beta_impute_raw[5,27]"
7,1.4982265,0.0069840063,0.22234685,1.061910e+00,1.343338e+00,1.5059335,1.6490110,1.9177824,1013.5676,0.9995534,"beta_impute_raw[4,4]"
8,0.5832399,0.0092835255,0.30322833,2.510396e-02,3.715368e-01,0.5748025,0.7738567,1.2013517,1066.8754,0.9990317,"beta_impute_raw[6,4]"
9,0.1147143,0.0017639371,0.05902539,5.266292e-03,7.296193e-02,0.1127369,0.1549204,0.2334397,1119.7256,0.9990989,"beta_impute[6,4]"


,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.51640867,0.011842587,0.41118668,-0.30030119,0.25400458,0.48908695,0.787742967,1.36391972,1205.549,1.003561,"beta_impute_raw[3,21]"
2,0.23161804,0.001574323,0.07256845,0.08728896,0.18406437,0.23234613,0.278254824,0.37151915,2124.750,1.003547,"beta_impute[1,9]"
3,0.10170174,0.002306241,0.08079381,-0.05972333,0.04961109,0.09612933,0.154734287,0.26551895,1227.289,1.003540,"beta_impute[3,21]"
4,-0.22552211,0.009534655,0.38590639,-0.98936137,-0.48054038,-0.23625986,0.036739931,0.49465571,1638.151,1.003409,"beta_impute_raw[6,21]"
5,-0.04452559,0.001859412,0.07585322,-0.19545827,-0.09451529,-0.04647340,0.007485014,0.09729545,1664.167,1.003375,"beta_impute[6,21]"
6,1.17621741,0.008417771,0.36959853,0.44055025,0.92927941,1.17041269,1.418843823,1.93140420,1927.819,1.002851,"beta_impute_raw[1,9]"
7,-0.11382482,0.006487158,0.28030502,-0.68483057,-0.29978539,-0.10539689,0.056003849,0.42525222,1867.037,1.002350,"beta_impute_raw[4,16]"
8,-0.02235305,0.001294869,0.05498725,-0.13486865,-0.05828562,-0.02075392,0.011042828,0.08294994,1803.319,1.002322,"beta_impute[4,16]"
9,0.79403343,0.008869983,0.38462335,0.06309948,0.53065739,0.77645881,1.070965202,1.53488170,1880.294,1.002314,"beta_impute_raw[5,24]"


### Stage 2 regression code

In [21]:
stage_2_regress_nu_1 = "
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;

    matrix[ N, N_var ] x_raw;

    int<lower=0,upper=1> y[N];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    
    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    real x_missing_mean[ N_miss ];
    real x_missing_sd[ N_miss ];
    
}
parameters {
    // Needed for imputation
    vector<upper=0>[ N_miss ] x_impute_raw;
    
    //regression on y per metabolite in x_raw columns
    vector[N_var] beta_x;
    vector[N_var] alpha_x;

    real<lower=0> sigma_beta_x;
    //real<lower=1> nu_x;

}
transformed parameters {
    // https://mc-stan.org/docs/2_18/stan-users-guide/vectors-with-varying-bounds.html
    vector<upper=0>[ N_miss ] x_impute;
    x_impute = x_impute_raw;
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_impute[i] = x_impute[i] + threshold[metab_col];
    }

}
model {
    matrix[N, N_var] x_merge;
    //copy matrix
    x_merge = x_raw;
    
    //fill in the values we are going to impute
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_merge[metab_row,metab_col] = x_impute[ i ];
    }
    
    x_impute ~ normal(x_missing_mean, x_missing_sd);
    
    sigma_beta_x ~ cauchy(0,1);
    //nu_x ~ gamma(2,0.1);
    //beta_x ~  student_t(nu_x,0,sigma_beta_x);
    beta_x ~ student_t(1,0,sigma_beta_x);
    alpha_x ~ normal(0,5);
    
    for(j in 1:N_var) {
        vector[N] x_col = x_merge[ , j];
        real beta = beta_x[j];
        real alpha = alpha_x[j];
        y ~ bernoulli_logit(alpha + beta * x_col);
    }
}

"

In [22]:
stage_2_regress_code_file = 'stage_2_regress_nu_1.stan'
write_lines(x = stage_2_regress_nu_1, path = stage_2_regress_code_file)
stage_2_regress_model = stan_model(file = stage_2_regress_code_file, verbose = FALSE)

DIAGNOSTIC(S) FROM PARSER:
Info:
Left-hand side of sampling statement (~) may contain a non-linear transform of a parameter or local variable.
If it does, you need to include a target += statement with the log absolute determinant of the Jacobian of the transform.
Left-hand-side of sampling statement:
    x_impute ~ normal(...)



In [23]:
###
# This function formats the data for
###

default_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

stage_2_regress_make_data = function(x_censored, y, IMPUTE_FUNC=default_impute, threshold=NULL, ...) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
                 
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            


    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    # make a vector with naive imputation values
    if( N_miss > 0 ) {
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    
        mean_sds = IMPUTE_FUNC(x_censored,x_missing,...)
        x_missing_mean = mean_sds$x_missing_mean
        x_missing_sd = mean_sds$x_missing_sd
    }
    
    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_missing_mean=x_missing_mean,
            x_missing_sd=x_missing_sd,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [24]:
stage_1_impute = function(x_censored, x_missing, stage_1_post=NULL) {
    N_miss = dim(x_missing)[1]
    print(N_miss)
    x_missing_mean = apply( stage_1_post$x_impute_mean, FUN=mean, MARGIN=2 )
    x_missing_sd = apply( stage_1_post$x_impute, FUN=sd, MARGIN=2 )
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

fancy_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    
    x_obs = out$df_sample[,metabolites]
    for(i in 1:N_miss) {
        row = x_missing[i,1]
        col = x_missing[i,2]
        x_missing_mean[i] = x_obs[row,col]
        x_missing_sd[i] = 0.001
    }
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

#stage_2_init = list(
#    x_impute_raw = stage_2_data$x_missing_init_raw
#)




In [41]:
stage_1_post = extract(stage_1_fit)
stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)

stage_2_init = replicate_init(
    list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    ), chains=4
)


In [42]:
str(stage_2_data)
str(stage_2_init)

List of 12
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] 0.347 0.223 1.455 0.363 0.717 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] 0.347 0.223 1.455 0.363 0.717 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_missing_mean    : num(0) 
 $ x_missing_sd      : num(0) 
 $ threshold         : Named num [1:27] -2.78 -2.96 -2.71 -2.27 -2.4 ...
  ..- attr(*, "names")= chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ y                 : num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
List of 4
 $ 1:List of 1
  ..$ x_impute_raw: num(0) 
 $ 2:List of 

In [43]:
#stage_2_data$threshold
sum(apply(x_censored,FUN=function(x) { sum(is.na(x))},MARGIN=2))
x_censored[ stage_2_data$x_missing %>% head ]
out$df_sample[ stage_2_data$x_missing %>% head ]
stage_2_data$x_missing_mean[1:5]

[1] 0

numeric(0)

numeric(0)

[1] NA NA NA NA NA

In [44]:
stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=4, chains=4, iter = 2000, control=list(adapt_delta=0.8))


Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”


In [45]:
df_sum = data.frame(summary(stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_2_fit)$summary)
df_sum %>% arrange(n_eff)

mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
-3.684591e+03,0.6919597650,10.92836648,-3.704561e+03,-3.691791e+03,-3.685283e+03,-3.677438e+03,-3.662243e+03,249.4301,1.0075786,lp__
3.701673e-02,0.0010983675,0.02087636,1.001489e-02,2.190425e-02,3.228707e-02,4.739434e-02,8.873501e-02,361.2553,1.0055290,sigma_beta_x
-3.690706e-02,0.0028101013,0.08579202,-2.880730e-01,-5.923018e-02,-1.461017e-02,6.952403e-03,8.000814e-02,932.0729,1.0047115,beta_x[10]
8.163233e-02,0.0031507742,0.11421517,-4.997290e-02,3.374977e-03,3.973069e-02,1.309516e-01,3.815873e-01,1314.0533,1.0011177,beta_x[25]
1.937263e-02,0.0017239912,0.06563251,-9.095580e-02,-1.403721e-02,8.464752e-03,4.150763e-02,1.900792e-01,1449.3323,1.0018890,beta_x[15]
3.795383e-02,0.0020954162,0.08199490,-7.598412e-02,-5.768315e-03,1.514486e-02,6.361922e-02,2.594244e-01,1531.2042,1.0015268,beta_x[2]
1.023518e-01,0.0032262376,0.12751235,-4.692613e-02,7.168873e-03,5.866114e-02,1.714921e-01,4.173958e-01,1562.1106,1.0004788,beta_x[26]
3.076878e-02,0.0017297848,0.07171117,-7.109060e-02,-7.417970e-03,1.237584e-02,5.082741e-02,2.306777e-01,1718.6582,0.9999894,beta_x[1]
3.135674e-02,0.0017247942,0.07400027,-8.115909e-02,-8.437152e-03,1.307390e-02,5.701572e-02,2.213681e-01,1840.7382,0.9997210,beta_x[19]


### Run both models

In [31]:
make_summary_table_beta = function(fit,metabolites) {
    df_summary = data.frame(summary(fit)$summary)
    post = rstan::extract(fit)
    #compute prob > 0 or prob < 0
    n_sample = dim(post$beta_x)[1]
    g0 = colSums( (post$beta_x > 0) ) / n_sample 
    l0 = colSums( (post$beta_x < 0) ) / n_sample
    df_prob = data.frame(G0=g0,L0=l0) %>% rowwise() %>% mutate(Max=max(G0,L0))
    beta_names = c()
    for(i in seq(0,dim(post$beta_x)[2])) {
        beta_names=c(beta_names,paste0('beta_x[',i,']'))
    }
    df_summary_beta = df_summary %>% mutate(Param = rownames(.)) %>% 
    filter(Param %in% beta_names) %>% cbind(Metabolite = metabolites, P_GT_LT_0=df_prob$Max) %>%
    mutate(Z=mean/sd) %>% arrange(-P_GT_LT_0, -abs(Z)) %>% select(Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0)
    return(df_summary_beta)   
}

In [32]:
run_bayes_model = function(df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=0.8,
                           max_treedepth=10) {
    x_censored = df_censored[, metabolites]
    y = df_censored$died_90_day
    
    stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
    stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=4
    )
    
    control = list(adapt_delta=adapt_delta,max_treedepth=max_treedepth)
    print("CONTROL")
    print(control)
    
    N_miss = stage_1_data$N_miss
    stage_1_fit = NULL
    stage_1_post = NULL
    if(N_miss > 0) {
        print("IMPUTING")
        stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=cores, chains=chains, iter = iter,
                             control = control )

        stage_1_post = extract(stage_1_fit)
    } else {
        print("NO MISING DATA SKIPPING IMPUTATION")
    }
    
    stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)
    stage_2_init = list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    )
    
    stage_2_init = replicate_init(
        list(
            x_impute_raw = stage_2_data$x_missing_init_raw
        ), chains=4
    )
    
    
    stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=cores, chains=chains, iter = iter, control = control )

    
    sum_table = make_summary_table_beta(stage_2_fit,metabolites)
    rownames(sum_table) = sum_table$Metabolite
    list( sum_table=sum_table, stage_1_fit=stage_1_fit, stage_2_fit=stage_2_fit)
}

In [46]:
make_summary_table_beta(stage_2_fit,metabolites)

Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
propyleneglycol,beta_x[24],0.666463581,0.18847030,0.29743133,0.6674550241,1.03712789,0.9996492,3.53617301,1.00000
glutamine,beta_x[12],0.370037154,0.17224195,0.01417031,0.3754840180,0.69083725,1.0005452,2.14835668,0.98700
tyrosine,beta_x[26],0.102351823,0.12751235,-0.04692613,0.0586611392,0.41739578,1.0004788,0.80268167,0.80425
pyruvate,beta_x[25],0.081632326,0.11421517,-0.04997290,0.0397306881,0.38158733,1.0011177,0.71472405,0.79250
oxoisocaproate,beta_x[2],0.037953831,0.08199490,-0.07598412,0.0151448563,0.25942443,1.0015268,0.46288038,0.67425
creatinine,beta_x[10],-0.036907057,0.08579202,-0.28807301,-0.0146101739,0.08000814,1.0047115,-0.43019217,0.66825
hydroxybutyrate,beta_x[1],0.030768784,0.07171117,-0.07109060,0.0123758371,0.23067768,0.9999894,0.42906542,0.66525
methionine,beta_x[19],0.031356736,0.07400027,-0.08115909,0.0130738993,0.22136810,0.9997210,0.42373814,0.65625
valine,beta_x[27],0.022620307,0.06577568,-0.08827544,0.0099378615,0.19329368,0.9999605,0.34390076,0.61800


In [48]:
metabolites[ out$truth ]

[1] "glutamine"       "propyleneglycol" "pyruvate"

In [56]:
bayes_model_res = run_bayes_model(out$df_censored,metabolites,cores=4,chains=4,iter=2500, 
                           adapt_delta=0.8,
                           max_treedepth=10)

[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


In [58]:
apply( out$df_censored, FUN=function(x) { mean(is.na(x)) }, MARGIN=1)

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[112] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[149] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[186] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [59]:
names(bayes_model_res)

[1] "sum_table"   "stage_1_fit" "stage_2_fit"

In [60]:
df_sum = data.frame(summary(bayes_model_res$stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(bayes_model_res$stage_2_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(20)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-3.683678e+03,1.120144018,12.55266602,-3.703952e+03,-3.692096e+03,-3.685171e+03,-3.677042e+03,-3.652466e+03,125.5811,1.016477,lp__
2,3.693646e-02,0.001237310,0.02230824,6.407813e-03,2.059618e-02,3.276166e-02,4.811752e-02,9.157177e-02,325.0674,1.004592,sigma_beta_x
3,3.644150e-01,0.008376350,0.18654637,1.111613e-03,2.457818e-01,3.788085e-01,4.920040e-01,7.070534e-01,495.9799,1.005471,beta_x[12]
4,1.819627e-03,0.005657447,0.14383301,-2.703320e-01,-9.423548e-02,-1.441899e-03,9.677211e-02,3.207559e-01,646.3626,1.004101,alpha_x[10]
5,9.860369e-02,0.003900649,0.12515729,-4.219634e-02,5.345307e-03,5.325866e-02,1.644005e-01,4.239194e-01,1029.5292,1.002216,beta_x[26]
6,8.399035e-02,0.003437815,0.11448213,-4.764850e-02,2.843611e-03,4.331811e-02,1.378443e-01,3.802580e-01,1108.9457,1.004884,beta_x[25]
7,2.170108e-03,0.004242961,0.14400597,-2.726091e-01,-9.682925e-02,2.700641e-03,9.954445e-02,2.788987e-01,1151.9217,1.003548,alpha_x[26]
8,7.261050e-03,0.004039911,0.14121390,-2.724627e-01,-8.823972e-02,6.775590e-03,1.011882e-01,2.865404e-01,1221.8314,1.004398,alpha_x[2]
9,-4.449396e-04,0.004232083,0.14972678,-2.860055e-01,-1.074532e-01,-3.959913e-03,1.040218e-01,2.860063e-01,1251.6722,1.004275,alpha_x[23]


In [61]:
bayes_model_res$sum_table

,Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
propyleneglycol,propyleneglycol,beta_x[24],0.6600653964,0.19212070,0.276352054,0.6559591343,1.04623266,1.0008128,3.43568080,0.9992
glutamine,glutamine,beta_x[12],0.3644150293,0.18654637,0.001111613,0.3788085037,0.70705336,1.0054714,1.95348225,0.9774
tyrosine,tyrosine,beta_x[26],0.0986036932,0.12515729,-0.042196344,0.0532586649,0.42391936,1.0022162,0.78783817,0.8022
pyruvate,pyruvate,beta_x[25],0.0839903529,0.11448213,-0.047648499,0.0433181113,0.38025800,1.0048840,0.73365472,0.7820
oxoisocaproate,oxoisocaproate,beta_x[2],0.0362323090,0.07846856,-0.067929043,0.0146233035,0.25814202,1.0034497,0.46174303,0.6734
creatinine,creatinine,beta_x[10],-0.0384354839,0.08209839,-0.261769856,-0.0133889508,0.06998817,1.0001163,-0.46816370,0.6728
hydroxybutyrate,hydroxybutyrate,beta_x[1],0.0329184487,0.07741618,-0.084638608,0.0144611941,0.23999163,1.0003383,0.42521404,0.6524
methionine,methionine,beta_x[19],0.0286791762,0.07045048,-0.077871319,0.0125490100,0.21824067,0.9999884,0.40708276,0.6480
valine,valine,beta_x[27],0.0253689927,0.07063005,-0.081158076,0.0094751249,0.20815456,1.0009527,0.35918127,0.6298


### Test Full Simulation

In [69]:
run_sim = function(n_sim=100,n_0=100,n_1=100,frac_sig=0.4,
                   metabolites=NULL,censor=TRUE,max_missing=0.4,iter=2000,sim_number=1,MISSING_RATE=NULL,alpha=NULL,beta=NULL) {
    res = data.frame(stringsAsFactors = F)
    for(i in 1:n_sim) {
        sim_data = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=frac_sig,censor=censor,max_missing=max_missing,alpha = alpha, beta=beta)
        df_censored = sim_data$df_censored
        df_naive_impute = sim_data$df_naive_impute
        truth = sim_data$truth
        
        out_sim = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_SIM_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_sim)
        write_rds(sim_data, out_sim, compress = 'gz')
        
        df_eff_sim = sapply(metabolites,FUN=function( x ) { apply_logistic_sim(x,df_naive_impute) } )
        sig_raw = as.vector( df_eff_sim['Pr(>|z|)',] < 0.05)
        eff_raw = as.vector( df_eff_sim['Estimate',])
        
        sig_bon = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='bonferroni') < 0.05)
        eff_bon = as.vector( df_eff_sim['Estimate',])
        
        sig_bh = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='BH') < 0.05)
        eff_bh = as.vector( df_eff_sim['Estimate',])
        
        bayes_model_res = run_bayes_model(df_censored,metabolites,cores=4,chains=4,iter=iter, 
                           adapt_delta=adapt_delta,
                           max_treedepth=max_treedepth)
        
        
        sum_table = bayes_model_res$sum_table
        stage_1_fit = bayes_model_res$stage_1_fit
        stage_2_fit = bayes_model_res$stage_2_fit
        
        out_fit = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_FIT_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_fit)
        write_rds(bayes_model_res, out_fit, compress = 'gz')

        sig_bayes = as.vector( sum_table[metabolites,'P_GT_LT_0'] > 0.975  )
        eff_bayes = as.vector( sum_table[metabolites,'mean'] )
        
        res_bayes = compute_stats(sig_bayes,truth,'bayes', eff_bayes, true_effs )
        res = rbind(res,res_bayes,stringsAsFactors = F)
        
        res_raw = compute_stats(sig_raw,truth,'raw', eff_raw, true_effs )
        res = rbind(res,res_raw,stringsAsFactors = F)
        res_bon = compute_stats(sig_bon,truth,'bon', eff_bon, true_effs )
        res = rbind(res,res_bon,stringsAsFactors = F)
        res_bh = compute_stats(sig_bh,truth,'bh', eff_bh, true_effs )
        res = rbind(res,res_bh,stringsAsFactors = F)
        
    }
    res
}


In [63]:
apply_logistic_sim = function(met,df) {
    m = glm(died_90_day ~ I(df[,met]) ,data=df,family="binomial")
    summary(m)$coef[2,]
}

compute_stats = function(sig, truth, label, est_effs, true_effs ) {
    tp = sum(   (sig == 1) & (truth == 1)  )
    fn = sum(  (sig == 0) & (truth == 1) )
    fp = sum(  (sig == 1) & (truth == 0)   )
    tn =  sum(  (sig == 0) & (truth == 0)  )
    avg_mag_error = mean( abs(est_effs[sig & truth]/true_effs[sig & truth]) )
    num_sign_error = sum( (sign(est_effs) != sign(true_effs))[sig & truth] )
    n_sig = sum(sig)
    n_true = sum(truth)
    
    if(n_sig == 0) {
        avg_mag_error = 0
        num_sign_error = 0
    }
    return(list(label=label,tp=tp,fn=fn,fp=fp,tn=tn,avg_mag_error=avg_mag_error, num_sign_error=num_sign_error, n_sig=n_sig, n_true=n_true))
}


In [64]:
sim_data = make_df(10000,mu_0,sigma_0,10000,mu_1,sigma_1,frac_sig=1,censor=FALSE)
df_sample = sim_data$df_sample
truth = sim_data$truth
df_eff_sim = sapply(colnames(df_sample)[1:27],FUN=function( x ) { apply_logistic_sim(x,df_sample) } )
true_effs = as.numeric(df_eff_sim['Estimate',])
true_effs

[1]  0.1537961576 -0.0794455222  0.0732114217  0.4871919864  0.6416351998
 [6]  0.3603286513  0.2211080879  0.2824915120  0.2472203365  0.3760155040
[11]  0.0009567894  0.5442929500  0.5700880170  0.3129254193  0.0808985480
[16]  0.5143552338  0.2065951259  0.5495977666  0.4901965309  0.7285902778
[21]  0.2992535715  0.3397080426  0.4882431039  0.6325416350  0.2435654628
[26]  0.6488578293  0.1659112488

In [70]:

n_sample = 100
frac_sig=0.1
max_missing=0.4
n_iter=3000
adapt_delta=0.8
max_treedepth=10
TARGET_MISSING_RATE = 0.05
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = TARGET_MISSING_RATE)
alpha = beta_params$alpha
beta = beta_params$beta
sim_number=1
res = run_sim(n_sim=5,n_0=n_sample,
              n_1=n_sample,frac_sig=frac_sig,metabolites=metabolites,
              max_missing=max_missing,iter=n_iter,sim_number=1,MISSING_RATE=TARGET_MISSING_RATE,
              alpha=alpha, beta=beta, censor=FALSE)


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_FIT_DATA.rds"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_FIT_DATA.rds"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_FIT_DATA.rds"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_FIT_DATA.rds"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess”


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.1_MISSING_RATE_0.05_FIT_DATA.rds"


In [72]:
res

,label,tp,fn,fp,tn,avg_mag_error,num_sign_error,n_sig,n_true
,<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>
1,bayes,3,0,0,24,1.2355991,0,3,3
2,raw,3,0,1,23,1.4373464,0,4,3
3,bon,3,0,0,24,1.4373464,0,3,3
4,bh,3,0,0,24,1.4373464,0,3,3
5,bayes,1,2,0,24,1.6671925,0,1,3
6,raw,2,1,3,21,1.3313695,0,5,3
7,bon,1,2,0,24,1.8749438,0,1,3
8,bh,1,2,0,24,1.8749438,0,1,3
9,bayes,2,1,0,24,0.7033192,0,2,3
